# DKTC 최종 모델 학습 및 제출 파일 생성 (개선된 데이터 사용)

## 1. 전체 데이터로 모델 재학습 및 예측

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from mecab_ko import Tagger
import numpy as np

print("데이터 로딩 중...")
# 데이터 로드 (v2 일반 대화 사용)
train_df = pd.read_csv('data/train.csv')
general_df_v2 = pd.read_csv('data/train_general_conversation_v2.csv')
test_df = pd.read_csv('data/test.csv')

# 전체 학습 데이터 합치기
df_full_train = pd.concat([train_df, general_df_v2], ignore_index=True)

# README에 명시된 클래스 번호에 맞게 직접 매핑
class_mapping = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}
df_full_train['class'] = df_full_train['class'].map(class_mapping)

print(f"전체 학습 데이터 크기: {len(df_full_train)}")
print("데이터 준비 완료!")

In [ ]:
# Tokenizer 정의
mecab_tagger = Tagger()
def mecab_tokenizer(text):
    parsed = mecab_tagger.parse(text)
    return [line.split('	')[0] for line in parsed.split('
') if '	' in line]

# TF-IDF 벡터화 (1,3-gram)
print("TF-IDF 벡터화 진행 중...")
tfidf_vectorizer = TfidfVectorizer(tokenizer=mecab_tokenizer, ngram_range=(1, 3), min_df=3, max_df=0.9)

# 전체 학습 데이터로 TF-IDF 학습 및 변환
X_full_train_tfidf = tfidf_vectorizer.fit_transform(df_full_train['conversation'])

# 테스트 데이터 변환
X_test_tfidf = tfidf_vectorizer.transform(test_df['text'])

print(f"벡터화 완료! Vocabulary size: {len(tfidf_vectorizer.vocabulary_)}")

In [ ]:
# 로지스틱 회귀 모델 학습
print("최종 모델 학습 중...")
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_full_train_tfidf, df_full_train['class'])
print("모델 학습 완료!")

# 테스트 데이터 예측
print("테스트 데이터 예측 중...")
test_predictions = lr_model.predict(X_test_tfidf)
print("예측 완료!")

## 2. 최종 제출 파일 생성

In [ ]:
# 제출용 데이터프레임 생성
submission_final = pd.DataFrame()
submission_final['idx'] = test_df['idx']
submission_final['class'] = test_predictions

# 결과 확인
display(submission_final.head())

# 파일로 저장
submission_final.to_csv('submission_final_v2.csv', index=False)

print("submission_final_v2.csv 파일이 성공적으로 생성되었습니다.")